In [2]:
import random
import time
import numpy as np
import torch 
import torch.optim as optim
from torchvision import transforms, datasets
import copy
from Bio import SeqIO
import argparse
from utils.bert import get_config, BertModel, set_learned_params, BertForMaskedLM, visualize_attention, show_base_PCA, fix_params
from module import Train_Module
from dataload import DATA, MyDataset 
import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.cluster import adjusted_rand_score
import os
import time
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, completeness_score, homogeneity_score
import torch.nn.functional as F
from sklearn.cluster import MiniBatchKMeans, KMeans, AgglomerativeClustering, SpectralClustering 
import itertools  
from dataload import *
from MLM_SFP import *

start...


In [4]:
def model_device(model):
    print("device: ", 'cuda')
    print('-----start-------')
    model.to('cuda')

    model = torch.nn.DataParallel(model) # make parallel
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    return model

In [5]:
config = get_config(file_path = "./RNA_bert_config.json")

config.hidden_size = config.num_attention_heads * config.multiple    

model = BertModel(config)
model = BertForMaskedLM(config, model)
model = model_device(model = model)

model.load_state_dict(torch.load('bert_mul_2.pth'))

device:  cuda
-----start-------


<All keys matched successfully>

In [6]:
model.eval()

DataParallel(
  (module): BertForMaskedLM(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(6, 120, padding_idx=0)
        (position_embeddings): Embedding(440, 120)
        (token_type_embeddings): Embedding(2, 120)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-5): 6 x BertLayer(
            (attention): BertAttention(
              (selfattn): BertSelfAttention(
                (query): Linear(in_features=120, out_features=120, bias=True)
                (key): Linear(in_features=120, out_features=120, bias=True)
                (value): Linear(in_features=120, out_features=120, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=120, out_features=120, bias=True)
                (LayerNorm): BertLayerNorm(

In [19]:
max_length = config.max_position_embeddings
def load_data_EMB(record):
    gapped_seqs = []
    seqs = []

    gapped_seq = str(record).upper()
    gapped_seq = gapped_seq.replace("T", "U")
    seq = gapped_seq.replace('-', '')
    if set(seq) <= set(['A', 'T', 'G', 'C', 'U']) and len(list(seq)) < max_length:
        seqs.append(seq)
        gapped_seqs.append(gapped_seq)
    gapped_seqs = np.tile(onehot_seq(gapped_seqs, max_length*5), (1, 1))
    family = np.tile(np.array([1]), 1)
    seqs_len = np.tile(np.array([len(i) for i in seqs]), 1)   
    k = 1   
    kmer_seqs = kmer(seqs, k)
    masked_seq, low_seq = mask(kmer_seqs, rate = 0, mag = 1)
    kmer_dict = make_dict(k)
    swap_kmer_dict = {v: k for k, v in kmer_dict.items()}
    masked_seq = np.array(convert(masked_seq, kmer_dict, max_length))
    low_seq = np.array(convert(low_seq, kmer_dict, max_length))

    return seqs, low_seq

In [31]:
tokens = torch.IntTensor(load_data_EMB('AAA')[1])

In [43]:
emb = model(tokens)[2].squeeze(0)[:3]
emb.shape

In [ ]:
# python MLM_SFP.py --pretraining bert_mul_2.pth --data_embedding sample/aln/sample.raw.fa --embedding_output ./test.txt --batch 40